In [ ]:
# gett a 5s excerpt of first test file
from pyannote.database import get_protocol, FileFinder
protocol = get_protocol('Debug.SpeakerDiarization.Debug', 
                        preprocessors={"audio": FileFinder()})

from pyannote.audio.core.io import Audio
audio = Audio(sample_rate=16000, mono="downmix")
file = next(protocol.test())

from pyannote.core import Segment
waveform, sample_rate = audio.crop(file, Segment(5, 10))

import torch
waveforms = torch.tensor(waveform)[None, :]

In [ ]:
# play the excerpt
from IPython.display import Audio as Play
Play(waveforms.squeeze(), rate=sample_rate, normalize=False, autoplay=True)

In [ ]:
# define a model that simply returns the waveform
from pyannote.audio.core.model import Model
class Passthrough(Model):
    def forward(self, waveforms):
        return waveforms
    
identity = Passthrough()

In [ ]:
# pass the waveform through this "identity" model
Play(identity(waveforms).squeeze(), rate=sample_rate, normalize=False, autoplay=True)

In [ ]:
# add one torch_audiomentations waveform transform to the model
from pyannote.audio.augmentation.registry import register_augmentation
from torch_audiomentations import Gain
gain = Gain(
    min_gain_in_db=-15.0,
    max_gain_in_db=5.0,
    p=0.5)
register_augmentation(gain, identity, when='input')

In [ ]:
# pass the waveform through the "augmented" model
Play(identity(waveforms).squeeze(), rate=sample_rate, normalize=False, autoplay=True)